Dataset link https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis

In [ ]:
#configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing data set

In [ ]:
#API to fetch dataset from kaggle
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis
!kaggle datasets download -d kazanova/sentiment140

twitter-entity-sentiment-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# extracting the compressed dataset (zip file)

from zipfile import ZipFile
dataset = "/content/twitter_dataset.zip"
dataset2 = "/content/sentiment140.zip"

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')
with ZipFile(dataset2, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted
The dataset is extracted


In [ ]:

import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data processing

In [ ]:
#loading dataset
df = pd.read_csv('/content/twitter_training.csv', encoding= 'ISO-8859-1')

In [ ]:
print(df.shape, df_test.shape)

(74681, 4) (999, 4)


In [ ]:
df.head(5)


,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [ ]:
df_test.head(5)

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tomâs great auntie as âHayley canât get out of bedâ and told to his grandma, who now thinks Iâm a lazy, terrible person ð¤£"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp Iâve had Madeleine McCann in my c...


In [ ]:
#adding column name
column_names = ['twitte_id', 'entity', 'sentiment', 'text']
df = pd.read_csv('/content/twitter_training.csv',names=column_names, encoding= 'ISO-8859-1')



In [ ]:
print(df.shape, df_test.shape)

(74682, 4) (1000, 4)


In [ ]:
df.head(5)

,twitte_id,entity,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


missing values

In [ ]:
#in training dataset
df.isnull().sum()

twitte_id      0
entity         0
sentiment      0
text         686
dtype: int64

In [ ]:
#drop null vauels row from training dataset

df = df.dropna()
df.isnull().sum()


twitte_id    0
entity       0
sentiment    0
text         0
dtype: int64

In [ ]:
#check distribution in training dataset
df['sentiment'].value_counts()

Negative      22358
Positive      20655
Neutral       18108
Irrelevant    12875
Name: sentiment, dtype: int64

In [ ]:
# prompt: put negative as -1 possitive as 1 neutral as 0 and irrevent as 2

df['sentiment'] = df['sentiment'].replace(['Negative', 'Positive', 'Neutral', 'Irrelevant'], [-1,0,1,2])

In [ ]:
df.head(5)

,twitte_id,entity,sentiment,text
0,2401,Borderlands,0,im getting on borderlands and i will murder yo...
1,2401,Borderlands,0,I am coming to the borders and I will kill you...
2,2401,Borderlands,0,im getting on borderlands and i will kill you ...
3,2401,Borderlands,0,im coming on borderlands and i will murder you...
4,2401,Borderlands,0,im getting on borderlands 2 and i will murder ...


In [ ]:
df['sentiment'].value_counts()

-1    22358
 0    20655
 1    18108
 2    12875
Name: sentiment, dtype: int64

In [ ]:
# prompt: drop all Irrelevent and neutral sentiment rows

#df = df[df['sentiment'] != "Neutral"]
#df['sentiment'].value_counts()

In [ ]:
#convert target 4 to 1
#df.replace({'target':{4:1}}, inplace=True)

In [ ]:
#df['target'].value_counts()

0--> Negative Text
1--> Possitive Text

**Stemming**

reducing word to its root word

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [ ]:
df['stemmed_content'] = df['text'].apply(stemming) #this takes time

In [ ]:
df.head(5)

,twitte_id,entity,sentiment,text,stemmed_content
0,2401,Borderlands,0,im getting on borderlands and i will murder yo...,im get borderland murder
1,2401,Borderlands,0,I am coming to the borders and I will kill you...,come border kill
2,2401,Borderlands,0,im getting on borderlands and i will kill you ...,im get borderland kill
3,2401,Borderlands,0,im coming on borderlands and i will murder you...,im come borderland murder
4,2401,Borderlands,0,im getting on borderlands 2 and i will murder ...,im get borderland murder


In [ ]:
#seperate tarin test
x = df['stemmed_content'].values
y = df['sentiment'].values

In [ ]:
print(x)

['im get borderland murder' 'come border kill' 'im get borderland kill'
 ... 'realiz window partit mac year behind nvidia driver idea notic'
 'realiz window partit mac like year behind nvidia driver car fuck idea ever notic'
 'like window partit mac like year behind driver idea notic']


In [ ]:
print(y)

[0 0 0 ... 0 0 0]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=0)

In [ ]:
print(x.shape, x_train.shape, x_test.shape)

(73996,) (59196,) (14800,)


In [ ]:
print(y.shape, y_train.shape, y_test.shape)

(73996,) (59196,) (14800,)


In [ ]:
#converting text data into numeric deta using vectorizor
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [ ]:
print(x_train)

  (1, 17612)	0.5590675500976836
  (1, 3900)	0.7071923310124466
  (1, 7023)	0.22151039880775678
  (1, 15196)	0.3718274123904595
  (2, 6336)	0.26094215939086474
  (2, 17917)	0.5116542746905801
  (2, 19350)	0.5422791960926931
  (2, 20756)	0.2590535496174394
  (2, 6665)	0.34092108068110555
  (2, 11016)	0.23309145965738085
  (2, 1319)	0.2704493289984998
  (2, 10292)	0.25542508331257535
  (3, 5511)	0.2075646613653707
  (3, 5834)	0.26199338021698065
  (3, 10802)	0.12299785561699285
  (3, 15726)	0.2762423276117984
  (3, 6876)	0.13114452996608053
  (3, 196)	0.1698915475042507
  (3, 12833)	0.30000125325645594
  (3, 18747)	0.14235532684683944
  (3, 16703)	0.2350619102697446
  (3, 3495)	0.19132599002686085
  (3, 11340)	0.1411484768295131
  (3, 13910)	0.1851562807815898
  (3, 2329)	0.20479751951251204
  :	:
  (59194, 15401)	0.23198759208541425
  (59194, 5810)	0.31206275977819053
  (59194, 11860)	0.1633461348940201
  (59194, 18226)	0.18520132770303538
  (59194, 19038)	0.14096896664238304
  (59194, 1

In [ ]:
print(x_test)

  (0, 20071)	0.4527667069934517
  (0, 16747)	0.3726648727902264
  (0, 15386)	0.5996343360236032
  (0, 9207)	0.4319416025129958
  (0, 6439)	0.3316448653403035
  (1, 18977)	0.31084387152712445
  (1, 18945)	0.2216912325219752
  (1, 18434)	0.1870281542489032
  (1, 16458)	0.36329710782617797
  (1, 13076)	0.18203357113773327
  (1, 11583)	0.2267365687854224
  (1, 11464)	0.23561849087068076
  (1, 11273)	0.19624607308423891
  (1, 10558)	0.2262205147378169
  (1, 9568)	0.40405967530761644
  (1, 9394)	0.2455517655715282
  (1, 8933)	0.2975097707581969
  (1, 8610)	0.24955702850748945
  (1, 4249)	0.1668338794150797
  (1, 3838)	0.23522446570327807
  (2, 17612)	0.8863741145790289
  (2, 7489)	0.46296968475726613
  (3, 18747)	0.15430156915628584
  (3, 18226)	0.1907494227202866
  (3, 16970)	0.24892405804844603
  :	:
  (14797, 15367)	0.22689860919569188
  (14797, 12697)	0.2258018748497916
  (14797, 12354)	0.2560672541150647
  (14797, 10802)	0.20217242147157835
  (14797, 7263)	0.49949781862480314
  (14797, 

Tarining Logistic regression model

Logistic Regression

In [ ]:
model = LogisticRegression(max_iter=1000)


In [ ]:
x_t = x_train.reshape(-1,1)

In [ ]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

Model evoluation

Accuracy socre

In [ ]:
#Accuracy on training data
x_train_predict = model.predict(x_train)
training_data_accuracy = accuracy_score(y_train, x_train_predict)
print('Accuracy on Training Data: ', training_data_accuracy)

Accuracy on Training Data:  0.8343975944320562


In [ ]:
#Accuracy on testing data
x_test_predict = model.predict(x_test)
testing_data_accuracy = accuracy_score(y_test, x_test_predict)
print('Accuracy on Testining Data: ', testing_data_accuracy)

Accuracy on Testining Data:  0.7750675675675676
